In [89]:
import requests
import json
import pandas as pd
from functools import reduce

URL = 'http://search:8983/solr/news/select'
def get_search_result(search_words):

    num_row = 100
    total = 1
    result_docs = []
    start = 0
    while total > start :
        params = {'debugQuery':'on','indent': 'on', 'q': 'title:"(' + search_words  + ')"~100 AND article:"' + search_words + '"', 'sort': 'datetime desc', 'wt': 'json', 'start':start, 'rows':num_row}
        res = requests.get(URL, params=params)
        response = json.loads(res.text)
        total = response["response"]["numFound"]
        docs = response["response"]["docs"]
        #print("total:%d, start at %d" %(total, start))
        start = start + num_row
        result_docs.append(docs)

    merged_result = reduce(lambda x,y: x+y,result_docs)
    return merged_result

In [90]:
from pandas.io.json import json_normalize
df_result = json_normalize(get_search_result('문재인'))

In [92]:
df_wo_mt=df_result[(df_result['oid']!='008')]

In [93]:
df_wo_mt['article']=df_wo_mt['article'].str.replace(r'\[[A-Za-z가-힣0-9\= ]+\]|\([A-Za-z가-힣0-9\= ]+\)|\<[A-Za-z가-힣0-9\= ]+\>','')

In [94]:
%%time
from konlpy.tag import Mecab
mecab = Mecab()
pos = lambda d: ['/'.join(p) for p in mecab.pos(d)]
def mecab_words(sentence):
    temp = sentence.split('▶')
    if(len(temp)>1):sentence = temp[0]

    temp = sentence.split('☞')
    if(len(temp)>1):sentence = temp[0]

    temp = sentence.split('ⓒ')
    if(len(temp)>1):sentence = temp[0]
        
        
    #word_list = pos(sentence)
    word_list = mecab.nouns(sentence)
    #return " ".join(word_list)
    return " ".join(filter(lambda x: len(x.strip())>1, word_list))
    #return filter(lambda x: len(x)>1, " ".join(word_list))

df_wo_mt['article_me_noun'] = df_wo_mt['article'].apply(mecab_words)

CPU times: user 19.4 s, sys: 28 ms, total: 19.5 s
Wall time: 19.5 s


In [95]:
%%time
from collections import Counter

r1 = df_wo_mt['article_me_noun'].tolist()
result = Counter(" ".join(r1).split(" "))


CPU times: user 276 ms, sys: 24 ms, total: 300 ms
Wall time: 297 ms


In [96]:
result.most_common(1000)

[('대표', 27342),
 ('문재인', 13097),
 ('민주당', 11930),
 ('후보', 11467),
 ('국민', 10193),
 ('대선', 8915),
 ('대통령', 7834),
 ('지사', 7001),
 ('기자', 5726),
 ('의원', 5704),
 ('시장', 4790),
 ('안희정', 4626),
 ('경선', 4351),
 ('정치', 4124),
 ('지지', 3949),
 ('조사', 3947),
 ('정부', 3814),
 ('우리', 3742),
 ('정권', 3731),
 ('주자', 3642),
 ('호남', 3502),
 ('생각', 3467),
 ('정당', 3384),
 ('이날', 3165),
 ('안철수', 3109),
 ('사람', 3043),
 ('문제', 2974),
 ('이재명', 2937),
 ('교체', 2870),
 ('탄핵', 2858),
 ('정책', 2851),
 ('서울', 2778),
 ('지지율', 2551),
 ('보수', 2526),
 ('총장', 2514),
 ('일자리', 2462),
 ('강조', 2429),
 ('박근혜', 2376),
 ('비판', 2361),
 ('자신', 2297),
 ('주장', 2290),
 ('때문', 2240),
 ('충남', 2179),
 ('개혁', 2178),
 ('경제', 2167),
 ('지역', 2165),
 ('선거', 2145),
 ('캠프', 2116),
 ('대한민국', 2113),
 ('관련', 2086),
 ('포인트', 2054),
 ('여론', 2047),
 ('가능', 1953),
 ('이번', 1922),
 ('발언', 1874),
 ('반기문', 1865),
 ('개헌', 1853),
 ('국회', 1839),
 ('광주', 1837),
 ('국가', 1775),
 ('세력', 1719),
 ('입장', 1693),
 ('성남', 1666),
 ('대행', 1642),
 ('참여', 1624),
 ('상황',